In [0]:
# Variáveis
database = "silver"
tabela = "transactions"
path = "pnbank.silver.transactions"


In [0]:
from pyspark.sql.functions import current_date, current_timestamp, expr, regexp_replace, trim, col

In [0]:
df_transactions = spark.sql(
f"""
WITH users AS (
    SELECT
        id AS id_users,
        yearly_income AS user_yearly_income,
        total_debt AS user_total_debt,
        credit_score AS user_credit_score
FROM
    pnbank.bronze.users_data
),

transactions AS (
    SELECT
        client_id AS id_client,
        card_id AS id_card,
        id AS id_transaction,
        date AS date_transaction,
        amount,
        use_chip
FROM
    pnbank.bronze.transactions_data
),

cards AS (
    SELECT
        id AS id_card,
        client_id AS id_client,
        card_type,
        card_on_dark_web
FROM
    pnbank.bronze.cards_data

)

SELECT
    CAST(tc.id_client AS INT),
    CAST(tc.id_transaction AS INT), 
    CAST(tc.date_transaction AS DATE),
    TRY_CAST(regexp_replace(tc.amount, '[$]', '') AS DECIMAL(10,2)) AS amount_transaction,
    c.card_type,
    tc.use_chip,
    TRY_CAST(regexp_replace(u.user_yearly_income, '[$]', '') AS DECIMAL(10,2)) AS user_yearly_income,
    TRY_CAST(regexp_replace(u.user_total_debt, '[$]', '') AS DECIMAL(10,2)) AS user_total_debt,
    CAST(u.user_credit_score AS INT),
    CAST(c.card_on_dark_web AS BOOLEAN)

FROM
    transactions tc
INNER JOIN
    users u
ON
    tc.id_client = u.id_users
INNER JOIN
    cards c
ON
    tc.id_card = c.id_card

"""
)

In [0]:
df_transactions = df_transactions.withColumn("data_carga", current_date()) 
df_transactions = df_transactions.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL - 3 HOURS"))


In [0]:
df_transactions.write\
  .mode('overwrite') \
  .option('overwriteSchema', 'true') \
  .saveAsTable(f'{path}')


In [0]:
display(df_transactions)


In [0]:
%sql
SELECT id_client, user_credit_score FROM pnbank.silver.transactions
WHERE id_client = 1556
ORDER BY date_transaction

In [0]:
%sql
SELECT COUNT(*) FROM pnbank.silver.transactions;
